# xMelts liquid Endmembers Standard State Code Generator 

A notebook to generate spud files for xMelts standard state 

Required system packages and initialization

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import time
sym.init_printing()

Required ENKI packages (using the ThermoCodegen version of the coder package)

In [ ]:
from thermocodegen.coder import coder

Get the working version of ThermoCodegen

In [ ]:
import thermocodegen as tcg
tcg_version = tcg.__version__
print('Using ThermoCodegen v{}'.format(tcg_version))

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../endmembers'
try:
    os.mkdir(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v{}/share/thermocodegen/examples/Systems/fo_fa/notebooks/Generate_xmelts_endmembers.ipynb'.format(tcg_version)

In [ ]:
df = pd.read_csv('data/xMelts.csv')
df.columns

In [ ]:
# pull out column names required for just standard state model
cols = ['Phase', 'Formula',  'H_TrPr', 'S_TrPr', 'V_TrlPr', 
           'T_fus', 'S_fus', 'Cpl',
           'k0', 'k1', 'k2', 'k3', 'v1', 'v2', 'v3', 'v4']

Rename some columns for use with spud files

In [ ]:
df = df[cols]
df = df.rename(index=str,columns={'Phase':'name', 'Formula':'formula'})

restrict to just Forsterite and Fayalite

In [ ]:
names = ['Forsterite', 'Fayalite']
df = df[df.name.isin(names)]
df.set_index('name')

### Generate Coder parameter tuples from the column headers

In [ ]:
param_strings = df.columns[2:]
param_strings = param_strings.insert(0,'Trl')
units = [ 'K','J', 'J/K', 'J/bar-m','K','J/K', 'J/K',
         'J/K-m','J-K^(1/2)-m','J-K/m' ,'J-K^2',
        '1/bar', '1/bar^2', '1/K',  '1/K^2']


params = coder.set_coder_params(param_strings, units)
print('parameter\tunits')
for p in params:
    print('{},\t\t{}'.format(p[0],p[1]))

#### extract sympy symbols pass them to the local dictionary for use in other sympy expressions

In [ ]:
symbol_dict = coder.get_symbol_dict_from_params(params)

# load local dictionary
locals().update(symbol_dict)


## initialize the coder model (Gibbs model)

In [ ]:
model= coder.StdStateModel.from_type()

Set up reference symbols (Trl already defined)

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

## Standard State Potentials

### Define model Potentials for the Standard State Potentials

An expression for the Gibbs free energy, $G(T,P)$ 

#### (1) $C_P$ integrals

The isobaric heat capacity terms parameterized as: 

$$
    C_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3, 
$$

and in addition the reference condition third law entropy, $S_{Tr,Pr}$, and enthalpy of formation from the
elements, $\Delta H_{Tr,Pr}$, constitute additional parameters:

In [ ]:
# Solid Heat Capacity
Cps = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3
Cps

In [ ]:
# Enthalpy and Entropy

In [ ]:
H = H_TrPr + sym.integrate(Cps,(T,Tr,T_fus))   + sym.integrate(Cpl,  (T,T_fus,T)) + T_fus*S_fus

In [ ]:
S = S_TrPr + sym.integrate(Cps/T,(T,Tr,T_fus)) + sym.integrate(Cpl/T,(T,T_fus,T)) + S_fus

In [ ]:
GPr = H  - T*S
GPr

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space


Liquid volume

In [ ]:
half = sym.Rational(1,2)
Vf = V_TrlPr + (T - Trl)*(v1 + half*v3*(P - Pr)) + (P - Pr)*(v2 + half*v4*(P - Pr))

In [ ]:
GPrToP = sym.integrate(Vf,(P,Pr,P))

#### Define standard state G and parameters

In [ ]:
G = GPr + GPrToP
G
G.simplify()

### Add the Gibbs Free Energy potential and parameters to the coder model

In [ ]:
model.add_potential_to_model('G',G, params)

Exam settable values for the model dictionary

In [ ]:
model.get_values()    

### Write out spud files for all xMelts standard state endmembers

In [ ]:
# loop over the rows of the dataframe
for i, row in df.iterrows():
    # create a dictionary from each row
    values_dict = row.to_dict()
    
    # clean up endmember names to make them coder compliant
    values_dict['name'] ='{}_xmelts'.format(values_dict['name'].title())
    values_dict['name'] = values_dict['name'].replace('-','_')
    print(values_dict['name'])
    
    # set additional parameters 
    values_dict['Trl'] = 1673.15
    values_dict['reference']=reference
    
    # update the coder model with specific values and write xml to the appropriate directory
    model.set_values(values_dict)
    model.to_xml(path=SPUD_DIR)
